# Low-rank approximation on $\mathcal{P}(d)$ - the space of $d$-dimensional SPD matrices

In this notebook we want to get some intuition in different approaches for computing low-rank approximations for manifold-valued signals

In [28]:
using Manifolds
using Manopt
using LinearAlgebra
using Random
using Plots
using LaTeXStrings

In [29]:
include("../../../src/decompositions/signals/naive_low_rank_approximation.jl")
include("../../../src/decompositions/signals/curvature_corrected_low_rank_approximation.jl")
include("../../../src/decompositions/signals/exact_low_rank_approximation.jl")

include("../../../src/functions/loss_functions/curvature_corrected_loss.jl")
include("../../../src/functions/loss_functions/exact_loss.jl")

exact_loss (generic function with 2 methods)

### Load data and construct manifold ###

In [30]:
# load data
M = SymmetricPositiveDefinite(3)
d = manifold_dimension(M)
n = 100  # 100


100

In [31]:
e = 1. * Matrix(I, 3, 3)
# compute basis
Θ = get_basis(M, e, DefaultOrthonormalBasis())
#  construct data
τ = 2.  # variance
σ = .05  # variance
Xₑ = Θ.data[4]
print(Xₑ)

Random.seed!(31)
predata = [exp(M, e, sqrt(τ) * randn(1)[1] * Xₑ) for i in 1:n]

data = [exp(M, predata[i], random_tangent(M, predata[i], Val(:Gaussian), σ)) for i in 1:n]; # ∈ P(3)^n


[0.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 0.0]

In [32]:
# Export slice image
num_export = 10
asymptote_export_SPD("results/artificial1D_orig.asy", data=data[1:min(num_export,n)], scale_axes=(2,2,2)); 

### Construct low rank approximation ###

In [33]:
q = mean(M, data)
log_q_data = log.(Ref(M), Ref(q), data);  # ∈ T_q P(3)^n

In [34]:
curvature_corrected_low_rank_approximation(M, q, data, 2) 

([[0.17325833245696642 -0.29091971605732453 0.14834305377860293; -0.2909197160573246 0.0010519744599409744 0.2618360748373959; 0.14834305377860296 0.2618360748373959 0.15467195813871742], [0.13312262520634632 0.08446349620388509 0.03350967638874231; 0.0844634962038851 -22.691052335418178 0.07537341502309187; 0.03350967638874231 0.07537341502309187 0.03380550088218992]], [0.04141185945717856 0.043589454988513694; -0.062377171919476435 0.06746220792835492; … ; 0.08428851980435229 -0.04426547676489212; 0.09977091731476738 0.09542655939149962])

In [41]:
exact_low_rank_approximation(M, q, data, 2; stepsize=1/10000, max_iter=50); 

Initial  F(x): 0.09762949978 | 
# 1     change: 0.043452491 |  F(x): 0.05179777061 | 
# 2     change: 0.059657190 |  F(x): 0.01594860649 | 
# 3     change: 0.063358610 |  F(x): 0.05660493647 | 
# 4     change: 0.058285768 |  F(x): 0.00991924308 | 
# 5     change: 0.055190020 |  F(x): 0.05294981081 | 
# 6     change: 0.059988626 |  F(x): 0.01597051307 | 
# 7     change: 0.063876396 |  F(x): 0.05776102151 | 
# 8     change: 0.058220309 |  F(x): 0.00903828027 | 
# 9     change: 0.053189782 |  F(x): 0.05157091431 | 
# 10    change: 0.060627068 |  F(x): 0.01840749903 | 
# 11    change: 0.065569353 |  F(x): 0.05766565645 | 
# 12    change: 0.058289291 |  F(x): 0.00921186403 | 
# 13    change: 0.053631637 |  F(x): 0.05192366074 | 
# 14    change: 0.060487520 |  F(x): 0.01782380249 | 
# 15    change: 0.065224761 |  F(x): 0.05775240393 | 
# 16    change: 0.058243411 |  F(x): 0.00906494541 | 
# 17    change: 0.053240378 |  F(x): 0.05158004482 | 
# 18    change: 0.060601070 |  F(x): 0.01832546872

In [36]:
max_iter = 50

nR_q = []
nU = []
ccR_q = []
ccU = []
eR_q = []
eU = []
for i in 1:d
    println("#$(i) | computing naive low-rank approximation")
    nRr_q, nUr = naive_low_rank_approximation(M, q, data, i)
    push!(nR_q, nRr_q)
    push!(nU, nUr)
    println("#$(i) | computing curvature corrected low-rank approximation")
    ccRr_q, ccUr = curvature_corrected_low_rank_approximation(M, q, data, i); 
    push!(ccR_q, ccRr_q)
    push!(ccU, ccUr)
    println("#$(i) | computing exact low-rank approximation")
    eRr_q, eUr = exact_low_rank_approximation(M, q, data, i; stepsize=1/10000, max_iter=max_iter); 
    push!(eR_q, eRr_q)
    push!(eU, eUr)
end

#1 | computing naive low-rank approximation
#1 | computing curvature corrected low-rank approximation
#1 | computing exact low-rank approximation
Initial  F(x): 0.13166108791 | 
# 1     change: 0.034606484 |  F(x): 0.10439405420 | 
# 2     change: 0.041838630 |  F(x): 0.06238753221 | 
# 3     change: 0.056285392 |  F(x): 0.00517685054 | 
# 4     change: 0.031123974 |  F(x): 0.03000510454 | 
# 5     change: 0.066681434 |  F(x): 0.04995588114 | 
# 6     change: 0.061229126 |  F(x): 0.02245472989 | 
# 7     change: 0.066218234 |  F(x): 0.05616634219 | 
# 8     change: 0.058886546 |  F(x): 0.01281543791 | 
# 9     change: 0.058485960 |  F(x): 0.05567113736 | 
# 10    change: 0.059066837 |  F(x): 0.01352187737 | 
# 11    change: 0.059560738 |  F(x): 0.05627005310 | 
# 12    change: 0.058830456 |  F(x): 0.01262467591 | 
# 13    change: 0.058155281 |  F(x): 0.05543876188 | 
# 14    change: 0.059138555 |  F(x): 0.01382516844 | 
# 15    change: 0.059972223 |  F(x): 0.05645306081 | 
# 16    chan

In [37]:
ref_distance = sum(distance.(Ref(M), Ref(q), data).^2)

naive_tangent_distances_r = zeros(d)
predicted_naive_distances_r= zeros(d)
true_naive_distances_r= zeros(d)

curvature_corrected_tangent_distances_r = zeros(d)
predicted_curvature_corrected_distances_r = zeros(d)
true_curvature_corrected_distances_r = zeros(d)

exact_tangent_distances_r = zeros(d)
exact_distances_r= zeros(d)

for rank in 1:d
    naive_log_q_data_r = Symmetric.([sum([nR_q[rank][i] * nU[rank][k,i] for i in 1:rank]) for k in 1:n])
    curvature_corrected_log_q_data_r = Symmetric.([sum([ccR_q[rank][i] * ccU[rank][k,i] for i in 1:rank]) for k in 1:n])
    exact_log_q_data_r = Symmetric.([sum([eR_q[rank][i] * eU[rank][k,i] for i in 1:rank]) for k in 1:n])
    
    # expoentiate back
    naive_data_r = exp.(Ref(M), Ref(q), naive_log_q_data_r)
    curvature_corrected_data_r = exp.(Ref(M), Ref(q), curvature_corrected_log_q_data_r)
    exact_data_r = exp.(Ref(M), Ref(q), exact_log_q_data_r)


    # compute relative tangent space error
    naive_tangent_distances_r[rank] = sum(norm.(Ref(M), Ref(q),  log_q_data - naive_log_q_data_r).^2) / ref_distance
    curvature_corrected_tangent_distances_r[rank] = sum(norm.(Ref(M), Ref(q),  log_q_data - curvature_corrected_log_q_data_r).^2) / ref_distance
    exact_tangent_distances_r[rank] = sum(norm.(Ref(M), Ref(q),  log_q_data - exact_log_q_data_r).^2) / ref_distance


    # compute relative manifold error
    predicted_naive_distances_r[rank] = curvature_corrected_loss(M, q, data, naive_log_q_data_r)
    true_naive_distances_r[rank] = exact_loss(M, q, data, naive_log_q_data_r)
    predicted_curvature_corrected_distances_r[rank] = curvature_corrected_loss(M, q, data, curvature_corrected_log_q_data_r)
    true_curvature_corrected_distances_r[rank] = exact_loss(M, q, data, curvature_corrected_log_q_data_r)
    exact_distances_r[rank] = exact_loss(M, q, data, exact_log_q_data_r)
end

In [38]:
# We want plots with (1) the lower bound error, (2) the actually uncorrected manifold error and (3) the corrected manifold error
plot(1:d, [naive_tangent_distances_r, true_naive_distances_r, true_curvature_corrected_distances_r, exact_distances_r], label = ["theoretical lower bound" "naive" "curvature corrected" "exact"], ylims=(0,1), xlims=(1,d),xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$"))
savefig("results/artificial1D_errors_by_rank.png")
plot(1:d, [naive_tangent_distances_r .+ 1e-16, true_naive_distances_r .+ 1e-16, true_curvature_corrected_distances_r .+ 1e-16, exact_distances_r .+ 1e-16], label = ["theoretical lower bound" "naive" "curvature corrected" "exact"], ylims=(1e-16,1), xlims=(1,d), xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$", :log), legend=:bottomleft)
savefig("results/artificial1D_logerrors_by_rank.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/1D/P3/results/artificial1D_logerrors_by_rank.png"

In [39]:
# It would be nice to also have a plot that tells us something about the error in predicting the manifold loss (using CCL) and the actual loss 
# (1) for the naive approach (2) for the curvature corrected approach
plot(1:d, [(predicted_naive_distances_r .+ 1e-16) ./ (true_naive_distances_r .+ 1e-16), (predicted_curvature_corrected_distances_r .+ 1e-16) ./ (true_curvature_corrected_distances_r .+ 1e-16)], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,d),xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$"))
savefig("results/artificial1D_discrepancy_by_rank.png")
plot(1:d, [(predicted_naive_distances_r .+ 1e-16) ./ (true_naive_distances_r .+ 1e-16), (predicted_curvature_corrected_distances_r .+ 1e-16) ./ (true_curvature_corrected_distances_r .+ 1e-16)], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,d), xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$", :log), legend=:bottomleft)
savefig("results/artificial1D_logdiscrepancy_by_rank.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/1D/P3/results/artificial1D_logdiscrepancy_by_rank.png"